## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('./energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

In [61]:
# 14일과 일부 15일의 데이터로 15일을 예측해야하므로 test_data 수정

test_data=data[(data['time'] >= '2023-10-14') & (data['time'] < '2023-10-16')].copy()
test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성

In [5]:
# 텐서로 변환해서 텐서데이터셋을 생성하세요. (1~6 번에 내용을 채워보세요.)

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = 24
batch_size = 32

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(train_data[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.tensor(trainX, dtype=torch.float32)
trainY_tensor = torch.tensor(trainY, dtype=torch.float32)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True)

### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [8]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)

input_size = 13   # 입력 특징 수 (단일 시계열)
hidden_size = 50 # LSTM의 은닉층 노드 수
num_layers = 2   # LSTM 레이어 수
learning_rate = 0.001
num_epochs = 20 # 학습 에폭 수

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # 최종 출력 레이어 (단일 값 예측)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 cell state

        out, _ = self.lstm(x, (h0, c0))  # LSTM 순전파
        out = self.fc(out[:, -1, :])     # 마지막 time-step의 출력만 사용
        return out

In [9]:
model = LSTMModel(input_size, hidden_size, num_layers)
criterion = nn.MSELoss()  # 손실 함수 (평균 제곱 오차)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 학습
for epoch in range(num_epochs):
    for batchX, batchY in train_loader:
        output = model(batchX)
        loss = criterion(output, batchY)

        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()        # 역전파
        optimizer.step()       # 가중치 업데이트

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/20], Loss: 0.0185
Epoch [20/20], Loss: 0.0064


In [58]:
X_test, _ = build_dataset(test_data[features + [target]].values, 24)

# Convert to PyTorch tensor
X_test_torch = torch.tensor(X_test, dtype=torch.float32)

# Set the model to evaluation mode
model.eval()

# Make prediction
with torch.no_grad():
    prediction = model(X_test_torch)

prediction_list = prediction.squeeze().tolist()
prediction_list = scaler_y.inverse_transform(np.array(prediction_list).reshape(-1, 1))

# Print the prediction result for 2023-10-15
print("Prediction for 2023-10-15:", prediction_list)

predicted_amounts = prediction_list.flatten().tolist()

Prediction for 2023-10-15: [[8.87527987e-02]
 [4.11139950e-02]
 [4.31229174e-02]
 [2.67191328e-01]
 [3.72990280e-01]
 [2.39345461e-01]
 [2.47231409e-01]
 [2.30027571e+00]
 [1.23345484e+01]
 [3.36018267e+01]
 [4.67211366e+01]
 [5.97959447e+01]
 [6.50161729e+01]
 [6.45917292e+01]
 [5.74136643e+01]
 [5.18486233e+01]
 [3.54812753e+01]
 [1.98530667e+01]
 [4.53292394e+00]
 [8.71671245e-01]
 [1.24358602e-01]
 [1.21724755e-01]
 [1.07423447e-01]
 [3.15346867e-02]]


In [19]:
filtered_data=data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy() # 시간 index 생성

### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [59]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=filtered_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [60]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

2023-10-15 예측 발전량 (24시간): [0.08875279873609543, 0.041113995015621185, 0.04312291741371155, 0.2671913281083107, 0.37299028038978577, 0.2393454611301422, 0.2472314089536667, 2.3002757132053375, 12.334548354148865, 33.601826667785645, 46.72113656997681, 59.795944690704346, 65.01617288589478, 64.59172916412354, 57.4136643409729, 51.848623275756836, 35.4812753200531, 19.85306668281555, 4.532923936843872, 0.8716712445020676, 0.12435860186815262, 0.12172475457191467, 0.10742344707250595, 0.03153468668460846] kWh


---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

In [ ]:
# transformer 모델은 피드포워드나 self-attention 과정에서 메모리를 많이 차지하고 계산이 너무 복잡하여 훈련에 큰 비용이 든다는 단점이 존재한다.
# Informer model은 transformer의 이러한 시간복잡도와 메모리 효율성을 개선하고자 했다.

### (필수) 모델링 해석

In [ ]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.

#transformer와 informer의 경우 LSTF문제에 적합하나 이번 문제는 긴 sequence를 예측해야하는 문제가 아니므로 비용적인 효율성을 위해 LSTM 모델을 선택하였다.

### (선택) 데이터 해석

In [ ]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등